# --- Day 14: Regolith Reservoir ---

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:
```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```
This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:

```
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```
Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```
The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```
After a total of five units of sand have come to rest, they form this pattern:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```
After a total of 22 units of sand:
```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```
Finally, only two more units of sand can possibly come to rest:
```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```
Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:
```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```
Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?

In [ ]:
import logging
import sys

sys.path.append("..")
from aoc import get_logger, get_data, timer, test

logger = get_logger("day14", logging.INFO)
data = get_data("day14")

animate = 0
if __name__ == "__main__":
    animate = 0.01

logger.info("initialization complete!")

## Solution Part One

In [ ]:
from IPython.display import clear_output
import time


class Cave(object):
    def __init__(self, rocks: list, sand_origin=None, animate=0):
        self.rocks: list = rocks
        self.sand_origin: tuple = (500, 0) if sand_origin is None else sand_origin
        self.animate = animate
        self.matrix = dict()
        self.sand = set()
        self.floor = 0
        self.frame = [sys.maxsize, -sys.maxsize, None, None]  # x-min, x-max, y-min, y-max
        self._place_rocks()

    def _place_rocks(self):
        for line in self.rocks:
            coords = [tuple(map(int, pair.split(","))) for pair in line.split(" -> ")]
            for i in range(len(coords) - 1):
                start, stop = coords[i], coords[i + 1]
                self.frame[0] = min(start[0], stop[0], self.frame[0])
                self.frame[1] = max(start[0], stop[0], self.frame[1])
                for x in range(min(start[0], stop[0]), max(start[0], stop[0]) + 1):
                    for y in range(min(start[1], stop[1]), max(start[1], stop[1]) + 1):
                        if y + 2 > self.floor:
                            self.floor = y + 1
                        self.matrix[(x, y)] = "#"
        self.frame[2] = self.sand_origin[1]
        self.frame[3] = self.floor + 1

    def _drop(self, x, y):
        below = [(x, y + 1), (x - 1, y + 1), (x + 1, y + 1)]  # down, down-left, down-right
        for space in below:
            if self.matrix.get(space, ".") == ".":
                return space
        return x, y

    def drop_sand(self):
        while True:
            x_from, y_from = self.sand_origin
            while True:
                x, y = self._drop(x_from, y_from)
                self.matrix.pop((x_from, y_from), None)
                self.matrix[(x, y)] = "o"
                if self.animate:
                    print(self)
                    time.sleep(self.animate)
                    clear_output(wait=True)

                if y == y_from:
                    break
                x_from, y_from = x, y

                if y > self.floor:
                    if self.animate:
                        print(self)
                        time.sleep(2)
                    return

    def __repr__(self):
        matrix = []
        for y in range(self.frame[2], self.frame[3]):
            row = []
            for x in range(self.frame[0], self.frame[1]):
                row.append(self.matrix.get((x, y), "."))
            matrix.append("".join(row))
        return "\n".join(matrix)

    def _count_sand(self) -> int:
        sand = 0
        for x, y in self.matrix.keys():
            if self.matrix[(x, y)] == "o":
                sand += 1
        return sand

    def units_of_sand(self):
        return self._count_sand() - 1  # adjust for falling sand block


@timer
def part1(input_):
    cave = Cave(rocks=input_.split("\n"), animate=animate)
    cave.drop_sand()
    return cave.units_of_sand()


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

# --- Part Two ---

You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a y coordinate equal to two plus the highest y coordinate of any point in your scan.

In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. (This is as if your scan contained one extra rock path like -infinity,11 -> infinity,11.) With the added floor, the example above now looks like this:
```
        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
<-- etc #################### etc -->
```
To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at 500,0, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after 93 units of sand come to rest:
```
............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
```
Using your scan, simulate the falling sand until the source of the sand becomes blocked. How many units of sand come to rest?

## Solution Part Two

In [ ]:
class Cave2(Cave):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def drop_sand(self):
        while True:
            x_from, y_from = self.sand_origin
            while True:
                x, y = self._drop(x_from, y_from)
                self.matrix.pop((x_from, y_from), None)
                self.matrix[(x, y)] = "o"
                if self.animate:
                    print(self)
                    time.sleep(self.animate)
                    clear_output(wait=True)

                if y == y_from or y == self.floor:
                    break
                x_from, y_from = x, y
            if y == self.sand_origin[1]:
                break

    def units_of_sand(self):
        return self._count_sand()


@timer
def part2(input_):
    cave = Cave2(rocks=input_.split("\n"), animate=animate)
    cave.drop_sand()
    return cave.units_of_sand()


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 2:", part2(data.input))

# Performance tuning
Just don't print anything.

In [ ]:
animate = 0